### Proximos passos: Organizar os dados para o fine tunning usando duas bases: compras e produtos

### Conclusão inicial: Para o caso de usar somente uma base o fine tunning funcionou bem, mas ainda temos que manter os metadados mesmo que já tenha sido fornecido durante o fine-tunning. Quero fazer o teste onde o nome da função é sempre o mesmo nos dados de treinamento.

In [1]:
from openai import OpenAI
import pandas as pd

In [2]:
client = OpenAI(api_key = "API-KEY")

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": "Say this is a test"}],
)
response.choices[0].message.content

# Modelo: gpt-3.5-turbo-1106

Nome da base: Base de dados de compras
- ID_compra: Identificador único para cada compra.
- ID_produto: Identificador único para cada produto comprado.
- ID_cliente: Identificador único para cada cliente que fez a compra.
- valor: Valor total da compra.

Contexto:
Esta base de dados contém informações sobre as compras realizadas. Cada linha representa uma compra, com detalhes como o produto comprado, o cliente que fez a compra e o valor total da compra.

Nome da base: Base de dados de produtos
- ID_produto: Identificador único para cada produto.
- nome_produto: Nome do produto.
- preco_unitario: Preço unitário do produto.

Contexto:
Esta base de dados contém informações sobre os produtos disponíveis para compra. Cada linha representa um produto, com detalhes como o nome e o preço unitário.

Nome da base: Base de dados de clientes
- ID_cliente: Identificador único para cada cliente.
- nome_cliente: Nome do cliente.
- cidade: Cidade onde o cliente reside.

Contexto:
Esta base de dados contém informações sobre os clientes. Cada linha representa um cliente, com detalhes como o nome e a cidade onde reside.


## Como será as roles para os metadados?
{"messages": [{"role": "system", "content": "Considere o metadados desta base, que inclui o contexto geral da base e uma explicação para cada variável contida:\n\nNome da base: Base de dados de compras\n\nContexto da base: Esta base de dados contém informações sobre as compras realizadas. Cada linha representa uma compra, com detalhes como o produto comprado, o cliente que fez a compra e o valor total da compra.\n\nExplicação de cada variável:\nID_compra: Identificador único para cada compra.\nID_produto: Identificador único para cada produto comprado.\nID_cliente: Identificador único para cada cliente que fez a compra.\nvalor: Valor total da compra."}, {"role": "user", "content": "Quais são os dados disponíveis na base de dados compras?"}, {"role": "assistant", "content": "Esta base de dados contém informações sobre as compras realizadas. Cada linha representa uma compra, com detalhes como o produto comprado, o cliente que fez a compra e o valor total da compra."}]}

## Prompt inicial para mandar para o chatGPT gerar o json para o fine tunning:

Considere o seguinte formato de json:

{"messages":[{"role":"system","content":"Você é um expert em criar códigos em Python para responder várias perguntas de negócio que um usuário está interresado em descobrir. Seu código deve receber a base compras e retornar o resultado na variável res. Use as bibliotecas pandas e numpy para criar essa função.

Considere o dataframe base_compras com as seguintes colunas: ['ID_compra', 'ID_produto', 'ID_cliente', 'valor'] com os seguintes metadados:

Contexto: Esta base de dados contém informações sobre as compras realizadas. Cada linha representa uma compra, com detalhes como o produto comprado, o cliente que fez a compra e o valor total da compra.

ID_compra: Identificador único para cada compra. ID_produto: Identificador único para cada produto comprado. ID_cliente: Identificador único para cada cliente que fez a compra. valor: Valor total da compra.

e o seguinte dataframe base_produtos com as seguintes colunas: ['ID_produto', 'nome_produto', 'preco_unitario'] com os seguintes metadados:

Contexto: Esta base de dados contém informações sobre os produtos disponíveis para compra. Cada linha representa um produto, com detalhes como o nome e o preço unitário.

ID_produto: Identificador único para cada produto. nome_produto: Nome do produto. preco_unitario: Preço unitário do produto.

Escreva uma função em Python chamada responder_pergunta que recebe dois argumentos base_compras e base_produtos. A função deve retornar a resposta para a pergunta do usuário. Seja claro e conciso ao escrever a função. A função não necessariamente usa todas as bases e sim somente as que são necessárias para responder as perguntas."},{"role":"user","content":"Qual é o número total de compras na base de dados?"},{"role":"assistant","content":"import pandas as pd\n\n# Função para carregar a base de dados\ndef carregar_base_dados():\n    return pd.read_csv('base_de_dados_compras.csv')"}]}

Me gere a mesma coisa para as seguintes perguntas que devem ser na role user e as funções que devem estar na role assisstante. A role system deve ser exatamente a mesma do formato fornecido.


### Substituir pela pergunta atual
    Qual é o nome de um produto específico?

### Substituir pela função em python atual
python

def responde_pergunta(base_produtos, ID_produto):
    produto = base_produtos[base_produtos['ID_produto'] == ID_produto]
    if not produto.empty:
        return f"O nome do produto {ID_produto} é '{produto['nome_produto'].values[0]}'"
    else:
        return "Produto não encontrado."

# Fazer o estudo com as seguintes perguntas (Pensar em mais perguntas):

- Qual é o nome de um produto específico?
- Qual é o preço unitário de um produto específico?
- Qual é o valor médio de todas as compras?
- Qual é o produto mais caro disponível?
- Quantos produtos diferentes estão disponíveis para compra?
- Qual é o cliente que mais gastou em compras?
- Qual é o produto mais vendido?
- Qual é o valor máximo gasto em uma única compra?
- Qual é o cliente que comprou mais produtos?
- Qual é o produto menos vendido?
- Qual é o cliente que fez a compra mais cara?

In [3]:
# Base de dados de compras
compras_data = {
    'ID_compra': [1, 2, 3, 4],
    'ID_produto': [101, 102, 101, 103],
    'ID_cliente': [201, 202, 203, 201],
    'valor': [50.00, 30.00, 40.00, 70.00]
}

compras_df = pd.DataFrame(compras_data)

In [7]:
compras_df.columns

Index(['ID_compra', 'ID_produto', 'ID_cliente', 'valor'], dtype='object')

In [4]:
# Base de dados de produtos
produtos_data = {
    'ID_produto': [101, 102, 103],
    'nome_produto': ['Produto A', 'Produto B', 'Produto C'],
    'preco_unitario': [25.00, 15.00, 35.00]
}

produtos_df = pd.DataFrame(produtos_data)

In [8]:
produtos_df.columns

Index(['ID_produto', 'nome_produto', 'preco_unitario'], dtype='object')

In [5]:
# Base de dados de clientes
clientes_data = {
    'ID_cliente': [201, 202, 203],
    'nome_cliente': ['Cliente X', 'Cliente Y', 'Cliente Z'],
    'cidade': ['Cidade A', 'Cidade B', 'Cidade C']
}

clientes_df = pd.DataFrame(clientes_data)

In [6]:
# Imprimir os dataframes
print("Base de dados de compras:")
print(compras_df)

print("\nBase de dados de produtos:")
print(produtos_df)

print("\nBase de dados de clientes:")
print(clientes_df)

Base de dados de compras:
   ID_compra  ID_produto  ID_cliente  valor
0          1         101         201   50.0
1          2         102         202   30.0
2          3         101         203   40.0
3          4         103         201   70.0

Base de dados de produtos:
   ID_produto nome_produto  preco_unitario
0         101    Produto A            25.0
1         102    Produto B            15.0
2         103    Produto C            35.0

Base de dados de clientes:
   ID_cliente nome_cliente    cidade
0         201    Cliente X  Cidade A
1         202    Cliente Y  Cidade B
2         203    Cliente Z  Cidade C




    Qual é o nome de um produto específico?

python

def responde_pergunta(base_produtos, ID_produto):
    produto = base_produtos[base_produtos['ID_produto'] == ID_produto]
    if not produto.empty:
        return f"O nome do produto {ID_produto} é '{produto['nome_produto'].values[0]}'"
    else:
        return "Produto não encontrado."

    Qual é o preço unitário de um produto específico?

python

def responde_pergunta(base_produtos, ID_produto):
    produto = base_produtos[base_produtos['ID_produto'] == ID_produto]
    if not produto.empty:
        return f"O preço unitário do produto {ID_produto} é R${produto['preco_unitario'].values[0]}"
    else:
        return "Produto não encontrado."

    Qual é o valor médio de todas as compras?

python

def responde_pergunta(base_compras):
    return f"O valor médio de todas as compras é R${base_compras['valor'].mean()}"

    Qual é o produto mais caro disponível?

python

def responde_pergunta(base_produtos):
    produto_mais_caro = base_produtos.loc[base_produtos['preco_unitario'].idxmax()]
    return f"O produto mais caro disponível é '{produto_mais_caro['nome_produto']}' com preço de R${produto_mais_caro['preco_unitario']}"

    Quantos produtos diferentes estão disponíveis para compra?

python

def responde_pergunta(base_produtos):
    return f"Existem {len(base_produtos)} produtos diferentes disponíveis para compra."


    Qual é o cliente que mais gastou em compras?

python

def responde_pergunta(base_compras):
    cliente_mais_gastador = base_compras.groupby('ID_cliente')['valor'].sum().idxmax()
    valor_gasto = base_compras.groupby('ID_cliente')['valor'].sum().max()
    return f"O cliente que mais gastou em compras foi {cliente_mais_gastador} com um total de R${valor_gasto}"

    Qual é o produto mais vendido?

python

def responde_pergunta(base_compras):
    produto_mais_vendido = base_compras['ID_produto'].value_counts().idxmax()
    return f"O produto mais vendido é o {produto_mais_vendido}"

    Qual é o valor máximo gasto em uma única compra?

python

def responde_pergunta(base_compras):
    valor_maximo = base_compras['valor'].max()
    return f"O valor máximo gasto em uma única compra foi R${valor_maximo}"

    Qual é o cliente que comprou mais produtos?

python

def responde_pergunta(base_compras):
    cliente_mais_compras = base_compras['ID_cliente'].value_counts().idxmax()
    total_compras = base_compras['ID_cliente'].value_counts().max()
    return f"O cliente que comprou mais produtos foi {cliente_mais_compras} com um total de {total_compras} compras"

    Qual é o produto menos vendido?

python

def responde_pergunta(base_compras):
    produto_menos_vendido = base_compras['ID_produto'].value_counts().idxmin()
    return f"O produto menos vendido é o {produto_menos_vendido}"

    Qual é o cliente que fez a compra mais cara?

python

def responde_pergunta(base_compras):
    cliente_compra_mais_cara = base_compras.loc[base_compras.groupby('ID_cliente')['valor'].idxmax()]['ID_cliente'].values[0]
    valor_compra_mais_cara = base_compras.loc[base_compras.groupby('ID_cliente')['valor'].idxmax()]['valor'].max()
    return f"O cliente que fez a compra mais cara foi {cliente_compra_mais_cara} com uma compra de R${valor_compra_mais_cara}"


In [ ]:
# Código para mandar os dados para a OpenAI
client.files.create(
  file=open("data_v2.json", "rb"),
  purpose="fine-tune"
)

In [ ]:
# Código para realizar o fine-tunning no modelo com os dados do passo anterior
responde_fine_tunning = client.fine_tuning.jobs.create(
  training_file="file-0hteG8LPfEeRb30XhQRT67xC", 
  model="gpt-3.5-turbo-1106"
)
responde_fine_tunning

In [ ]:
responde_fine_tunning.id

In [ ]:
responde_fine_tunning.fine_tuned_model

In [ ]:
client.fine_tuning.jobs.retrieve("ftjob-KqnpXaaL3JCh1QKFWSqEFDHP")

In [ ]:
#client.fine_tuning.jobs.list(limit=10)

In [ ]:
model_id = "ft:gpt-3.5-turbo-1106:personal::8uQhfXnH"

In [ ]:
response = client.chat.completions.create(
    model=model_id,
    messages=[{"role": "user", "content": "Me fale sobre a Base de dados de clientes"}],
)
response.choices[0].message.content

In [ ]:
response = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": """Você é um expert em criar códigos em Python para responder várias perguntas de negócio
        que um usuário está interresado em descobrir. Seu código deve receber a base compras e retornar o resultado na 
        variável res. Use as bibliotecas pandas e numpy para criar essa função.
        Escreva uma função em Python chamada responder_pergunta que recebe um argumento chamado df. 
        A função deve retornar a resposta para a pergunta fornecida, utilizando os dados da base correspondente.
        Seja claro e conciso ao escrever a função."""},
        {"role": "user", "content": "Qual foi o terceiro produto mais vendido?"}],
)

In [ ]:
response = client.chat.completions.create(
    model=model_id,
    messages=[
        {"role": "system", "content": """Você é um expert em criar códigos em Python para responder várias perguntas de negócio
        que um usuário está interresado em descobrir. Seu código deve receber a base compras e retornar o resultado na 
        variável res. Use as bibliotecas pandas e numpy para criar essa função.
        Escreva uma função em Python chamada responder_pergunta que recebe um argumento chamado df. 
        A função deve retornar a resposta para a pergunta fornecida, utilizando os dados da base correspondente.
        Seja claro e conciso ao escrever a função."""},
        {"role": "user", "content": "Qual foi a cidade onde teve mais vendas?"}],
)

In [ ]:
print(response.choices[0].message.content)

In [ ]:
eval(response.choices[0].message.content)

In [ ]:
compras_df